In [ ]:
pip install cryptography tenseal matplotlib gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import gradio as gr
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import base64
import os
import tenseal as ts
import matplotlib.pyplot as plt

# Ensure NLTK's VADER lexicon is downloaded
try:
    nltk.data.find('sentiment/vader_lexicon')
except LookupError:
    nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# AES Encryption/Decryption Functions
def aes_encrypt(message, aes_key):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv))
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(message.encode()) + encryptor.finalize()
    return base64.b64encode(iv + ciphertext).decode()

def aes_decrypt(ciphertext, aes_key):
    data = base64.b64decode(ciphertext)
    iv = data[:16]
    cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv))
    decryptor = cipher.decryptor()
    return (decryptor.update(data[16:]) + decryptor.finalize()).decode()

# Bullying Detection
def contains_bullying(message):
    sentiment = sia.polarity_scores(message)
    return sentiment['neg'] > 0.5  # Negativity threshold

# Message Logging Functions
message_log = []

def log_message(sender, plaintext, encrypted_message, bullying_detected):
    message_log.append({
        "Sender": sender,
        "Plaintext": plaintext,
        "Encrypted": encrypted_message,
        "Bullying": "Yes" if bullying_detected else "No"
    })

def view_user_log():
    if not message_log:
        return "No messages logged yet!"
    return "\n".join([f"{msg['Sender']} -> Message: {msg['Plaintext']}" for msg in message_log])

def view_admin_log():
    if not message_log:
        return "No messages logged yet!"
    return "\n".join([f"{msg['Sender']} -> Encrypted: {msg['Encrypted']} | Bullying Alert: {msg['Bullying']}" for msg in message_log])

# Homomorphic Encryption Setup
def create_context():
    context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
    context.global_scale = 2**40
    return context

def encrypt_sentiment_scores(text, context):
    sentiment = sia.polarity_scores(text)
    return {key: ts.ckks_vector(context, [value]) for key, value in sentiment.items()}

def aggregate_sentiment_scores(encrypted_scores_list):
    aggregated_scores = encrypted_scores_list[0]
    for scores in encrypted_scores_list[1:]:
        for key in scores:
            aggregated_scores[key] += scores[key]
    return aggregated_scores

def decrypt_aggregated_scores(aggregated_scores):
    return {key: score.decrypt()[0] for key, score in aggregated_scores.items()}

# Visualization Function
def visualize_sentiment(analytics):
    plt.figure(figsize=(8, 4))
    plt.bar(analytics.keys(), analytics.values(), color=['green', 'blue', 'orange', 'red'])
    plt.title("Sentiment Analysis Results")
    plt.ylabel("Scores")
    plt.xlabel("Categories")
    plt.tight_layout()
    plt.savefig("sentiment_chart.png")
    return "sentiment_chart.png"

# Bullying Summary Function
def summarize_bullying():
    total_messages = len(message_log)
    bullying_count = sum(1 for msg in message_log if msg["Bullying"] == "Yes")
    return f"Total Messages: {total_messages}\nMessages with Bullying: {bullying_count}"

# Gradio App with Customized UI
aes_key = os.urandom(32)
context = create_context()

def send_message(sender, message):
    encrypted_message = aes_encrypt(message, aes_key)
    bullying_detected = contains_bullying(message)
    log_message(sender, message, encrypted_message, bullying_detected)
    return f"Message sent by {sender}!"

def analyze_sentiments():
    if not message_log:
        return "No messages logged yet!", None

    encrypted_scores_list = [encrypt_sentiment_scores(msg["Plaintext"], context) for msg in message_log]
    aggregated_scores = aggregate_sentiment_scores(encrypted_scores_list)
    decrypted_scores = decrypt_aggregated_scores(aggregated_scores)
    sentiment_chart = visualize_sentiment(decrypted_scores)
    return decrypted_scores, sentiment_chart

def app():
    with gr.Blocks(css="""
        .container {background-color: #f0f4f8;}
        .textbox, .button, .label, .dropdown {border-radius: 8px;}
        .button {background-color: #0084ff; color: white;}
        footer {font-size: 16px; font-weight: bold; text-align: center; margin-top: 20px;}
    """) as gui:
        gr.Markdown("## Secure Group Messaging System")

        with gr.Tab("User Page"):
            gr.Markdown("### User Chat")
            with gr.Row():
                sender = gr.Dropdown(["User1", "User2", "User3", "User4", "User5"], label="Sender")
                message = gr.Textbox(lines=2, placeholder="Type your message here...", label="Message")
                send_button = gr.Button("Send Message")
                feedback = gr.Textbox(label="Feedback", interactive=False)
            with gr.Row():
                view_user_log_button = gr.Button("View Message Log")
                user_message_log_display = gr.Textbox(label="User Message Log", interactive=False)

            send_button.click(send_message, inputs=[sender, message], outputs=[feedback])
            view_user_log_button.click(view_user_log, outputs=user_message_log_display)

        with gr.Tab("Admin Page"):
            gr.Markdown("### Admin Dashboard")
            with gr.Row():
                view_admin_log_button = gr.Button("View Encrypted Message Log")
                admin_message_log_display = gr.Textbox(label="Admin Encrypted Log", interactive=False)
            with gr.Row():
                analyze_button = gr.Button("Analyze Sentiments")
                sentiment_results = gr.Textbox(lines=5, label="Sentiment Results", interactive=False)
                sentiment_chart = gr.Image(label="Sentiment Chart")
            with gr.Row():
                bullying_summary_button = gr.Button("Summarize Bullying Alerts")
                bullying_summary_display = gr.Textbox(label="Bullying Summary", interactive=False)

            view_admin_log_button.click(view_admin_log, outputs=admin_message_log_display)
            analyze_button.click(analyze_sentiments, outputs=[sentiment_results, sentiment_chart])
            bullying_summary_button.click(summarize_bullying, outputs=bullying_summary_display)

        gr.Markdown("Developed by Gaurav Sharan, VIT University, Roll No: 20MIA1081")

    return gui

# Launch the App
app().launch()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://adff34837c1cf0da62.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
